In [1076]:
from github import Github,Auth
from IPython.display import HTML
from decouple import config
from jinja2 import Environment, FileSystemLoader


In [1077]:
g = Github(auth=Auth.Token(config("GITHUB_TOKEN")))

In [1078]:
g.get_user().name

'Giorgio Caizzi'

In [1079]:
g.get_user().get_repos().totalCount

18

In [1080]:
repos = [repo for repo in g.get_user().get_repos() if repo.name in ["pysurfline","py3dinterpolations","repomanager"]]

In [1081]:
len(repos)

3

In [1082]:
repos[0].get_issues().totalCount

9

In [1083]:
repos[0].get_workflows()[0].__dict__

{'_requester': <github.Requester.Requester at 0x10d047450>,
 '_id': <github.GithubObject._ValuedAttribute at 0x10d03f350>,
 '_name': <github.GithubObject._ValuedAttribute at 0x10d02c310>,
 '_path': <github.GithubObject._ValuedAttribute at 0x10d02e110>,
 '_state': <github.GithubObject._ValuedAttribute at 0x10d02d5d0>,
 '_created_at': <github.GithubObject._ValuedAttribute at 0x10c415fd0>,
 '_updated_at': <github.GithubObject._ValuedAttribute at 0x10cc4ce10>,
 '_url': <github.GithubObject._ValuedAttribute at 0x10cfab250>,
 '_html_url': <github.GithubObject._ValuedAttribute at 0x10c383a10>,
 '_badge_url': <github.GithubObject._ValuedAttribute at 0x10d02e390>,
 '_headers': {'server': 'GitHub.com',
  'date': 'Thu, 09 Nov 2023 00:14:55 GMT',
  'content-type': 'application/json; charset=utf-8',
  'transfer-encoding': 'chunked',
  'cache-control': 'private, max-age=60, s-maxage=60',
  'vary': 'Accept, Authorization, Cookie, X-GitHub-OTP, Accept-Encoding, Accept, X-Requested-With',
  'etag': 'W/

In [1084]:
repo_contents = [
    {
        "Name": repo.name,
        "Language": repo.language,
        "Description": repo.description,
        "Url": repo.html_url,
        "Stars": repo.stargazers_count,
        "Issues": {
            "Open": sum([issue.state == "open" for issue in repo.get_issues()]),
            "Total": repo.get_issues().totalCount,
        },
        "Pull Requests": {
            "Open": sum([issue.state == "open" for issue in repo.get_pulls()]),
            "Total": repo.get_pulls().totalCount,
        },
        "Branches": {
            "Default": repo.default_branch,
            "Total": repo.get_branches().totalCount,
            "Branches": [branch.name for branch in repo.get_branches()],
        },
        "Forks": {
            "Total": repo.forks_count,
            "Forks": [fork.full_name for fork in repo.get_forks()],
        },
        "Pages": repo.has_pages,
        "Collaborators": {
            "Total": repo.get_collaborators().totalCount,
            "Collaborators": [
                collaborator.login for collaborator in repo.get_collaborators()
            ],
        },
        "Workflows": {
            "Total": repo.get_workflows().totalCount,
            "Workflows": [workflow.name for workflow in repo.get_workflows()]
        },
    }
    for repo in repos
]

In [1085]:
# Set up the environment
env = Environment(loader=FileSystemLoader('.'),extensions=['jinja2.ext.loopcontrols'])
template = env.get_template('template.html')

In [1086]:
# html = template.render(repos=[vars(repo) for repo in repos])
html = template.render(repos=repo_contents)

In [1087]:
HTML(html)

In [1088]:
with open('index.html', 'w') as file:
    file.write(html)